In [1]:
import os
os.chdir("../src/")

In [2]:
import import_inpi as inpi
import transform_rcs as trcs
import pandas as pd

In [ ]:
inpi.import_inpi()

In [ ]:
df_rep_2018, df_pm_2018  = trcs.transform_rcs("2018")

In [ ]:
df_rep_2019, df_pm_2019 = trcs.transform_rcs("2019")

In [ ]:
df_rep_2020, df_pm_2020  = trcs.transform_rcs("2020")

In [ ]:
start_url = "http://data.cquest.org/inpi_rncs/imr/stock/2017/"
inpi.find_links(start_url)

In [3]:
df_rep_2017, df_pm_2017  = trcs.transform_rcs("2017")

/home/coder/work/data_inpi/src/transform_rcs.py:22: DtypeWarning: Columns (7,15,17,39,40,42) have mixed types. Specify dtype option on import or set low_memory=False.
  datainpi = pd.read_csv(zip_file.open(myfile), sep=";")
/home/coder/work/data_inpi/src/transform_rcs.py:22: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  datainpi = pd.read_csv(zip_file.open(myfile), sep=";")
/home/coder/work/data_inpi/src/transform_rcs.py:22: DtypeWarning: Columns (7,17,32,34) have mixed types. Specify dtype option on import or set low_memory=False.
  datainpi = pd.read_csv(zip_file.open(myfile), sep=";")
/home/coder/work/data_inpi/src/transform_rcs.py:22: DtypeWarning: Columns (7,15,17,25,32,34,35,37,40) have mixed types. Specify dtype option on import or set low_memory=False.
  datainpi = pd.read_csv(zip_file.open(myfile), sep=";")
/home/coder/work/data_inpi/src/transform_rcs.py:22: DtypeWarning: Columns (7,26,32) have mixed types. Specify dtype 

Taille des tables
134
134
taille des tables
(9504907, 48)
(4296552, 29)


In [4]:
df_rep_2017.head()


,Code Greffe,Nom_Greffe,Numero_Gestion,Siren,Type,Nom_Patronymique,Nom_Usage,Pseudonyme,Prénoms,Dénomination,...,Rep_Perm_Pays,Conjoint_Collab_Nom_Patronym,Conjoint_Collab_Nom_Usage,Conjoint_Collab_Pseudo,Conjoint_Collab_Prénoms,Conjoint_Collab_Date_Fin,ID_Représentant,date_greffe,Libelle_Evt,source
0,605,Nice,2017B01042,829319599,P.Physique,MEZNI,NaN,NaN,Gaith,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1,2017-05-02,Nouveau dirigeant,0605_S1_20170504_5_rep.csv
1,605,Nice,1994B00079,393589080,P.Physique,HATTU FREDERIQUE MARIE COLETTE,NaN,NaN,,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,11,2017-05-03,Nouveau dirigeant,0605_S1_20170504_5_rep.csv
2,605,Nice,2001D00252,435261052,P.Physique,BAMBINA,NaN,NaN,Richard,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,11,2017-04-27,Nouveau dirigeant,0605_S1_20170504_5_rep.csv
3,605,Nice,2001D00252,435261052,P.Physique,BAMBINA,NaN,NaN,Richard,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,11,2017-04-27,Nouveau dirigeant,0605_S1_20170504_5_rep.csv
4,605,Nice,2001D00252,435261052,P.Physique,PIETRI,NaN,NaN,Caroline Yvonne Marie,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,12,2017-04-27,Nouveau dirigeant,0605_S1_20170504_5_rep.csv
